In [2]:
!git clone https://github.com/jezknee/StartupsGame.git

Cloning into 'StartupsGame'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 262 (delta 167), reused 166 (delta 76), pack-reused 0 (from 0)
Receiving objects: 100% (262/262), 2.54 MiB | 25.74 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [3]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the directory containing your .py files to the Python path
# Assuming the cloned repository is named 'StartupsGame' and is in the current directory
repo_path = os.path.join(current_dir, 'StartupsGame')
sys.path.append(repo_path)

print(f"Added {repo_path} to the system path.")

Added /content/StartupsGame to the system path.


In [5]:
from ai_agent import Agent
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
import traceback
import sys
import startups_AI_game as sg
import startups_RL_environment as sr
import pandas as pd
from datetime import datetime
import time
import os

def plotLearning(x, scores, eps_history, filename):
    print("Creating plot...")
    try:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

        # Plot scores
        ax1.plot(x, scores, 'b-', alpha=0.7, label='Score')
        ax1.set_xlabel('Episode')
        ax1.set_ylabel('Score')
        ax1.set_title('Training Scores')
        ax1.grid(True)
        ax1.legend()

        # Plot epsilon values
        ax2.plot(x, eps_history, 'r-', alpha=0.7, label='Epsilon')
        ax2.set_xlabel('Episode')
        ax2.set_ylabel('Epsilon')
        ax2.set_title('Epsilon Decay')
        ax2.grid(True)
        ax2.legend()

        plt.tight_layout()
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Plot saved as {filename}")
        plt.show()
    except Exception as e:
        print(f"Error in plotting: {e}")
        traceback.print_exc()

print("Script starting...")



if __name__ == '__main__':
    print("Main block entered")
    default_companies = [["Giraffe Beer", 5],["Bowwow Games",6],["Flamingo Soft",7],["Octo Coffee", 8],["Hippo Powertech", 9],["Elephant Mars Travel", 10]]
    player_actions_pick_up = ["pickup_deck", "pickup_market"]
    player_actions_put_down = ["putdown_shares", "putdown_market"]

    try:
        print("Creating environment...")
        env = sr.StartupsEnv(total_players=4, num_humans=0, default_company_list=default_companies)
        print(f"Environment created successfully. Action space: {env.action_space}, Observation space: {env.observation_space}")


        print("Creating agent...")
        # make input_dims match the observation space without hardcoding
        agent = Agent(alpha=0.0005, gamma=0.99, n_actions=env.action_space.n, epsilon=1.0, batch_size=64, input_dims=env.observation_space.shape[0], epsilon_dec=0.996, epsilon_end=0.01, mem_size=1000000, fname='/content/startup_model4.keras')
        print("Agent created successfully")

        game_history = []
        action_history = []

        scores = []
        eps_history = []
        num_episodes = 1000

        for i in range(num_episodes):
            print(f"Starting episode {i}")
            done = False
            score = 0
            episode_start_time = time.time()  # Add this line

            try:
                observation, info = env.reset()
                print(f"Episode {i} - Initial observation shape: {observation.shape}")
            except Exception as e:
                print(f"Error during env.reset(): {e}")
                traceback.print_exc()
                continue

            step_count = 0
            rl_actions_taken = 0
            max_steps = 50

            while not done and step_count < max_steps:
                try:
                    current_phase = env.state_controller.get_current_phase()
                    print(f"Step {step_count}, Phase: {current_phase}")

                    if current_phase in (sr.TurnPhase.RL_PUTDOWN, sr.TurnPhase.RL_PICKUP):
                        action = agent.choose_action(observation, env)
                        print(f"RL agent choosing action {action}")
                        rl_actions_taken += 1

                        observation_, reward, terminated, truncated, info = env.step(action)
                        done = terminated or truncated

                        if 'invalid_action' in info and info['invalid_action']:
                            print(f"Invalid action taken: {action}")

                        agent.remember(observation, action, reward, observation_, done)
                        observation = observation_
                        score += reward

                        if agent.memory.mem_cntr > agent.batch_size:
                            agent.learn()

                        print(f"Action {action}, Reward: {reward}, Score: {score}")

                    else:
                        # Not RL agent's turn - step anyway to advance game state
                        # Pass a dummy action (0) since other players will be handled internally
                        observation_, reward, terminated, truncated, info = env.step(0)
                        done = terminated or truncated
                        observation = observation_
                        print(f"Other players' turn, game state advanced")

                    step_count += 1

                except Exception as e:
                    print(f"Error during step {step_count} of episode {i}: {e}")
                    traceback.print_exc()
                    break

            if step_count >= max_steps:
                print(f"Episode {i} exceeded {max_steps} steps, ending...")

            episode_runtime = time.time() - episode_start_time
            game_history.append({
                'episode': i,
                'score': score,
                'runtime': episode_runtime,
                'steps': step_count,
                'rl_actions': rl_actions_taken,
                'epsilon': agent.epsilon,
                'rl_rank': env._calculate_player_rank()+1,
                'rl_coins': env._get_coins_for_score()
            })
            eps_history.append(agent.epsilon)
            scores.append(score)

            avg_score = np.mean(scores[max(0, i-100):i+1])
            print(f'episode {i}, score {score:.2f}, average score {avg_score:.2f}, epsilon {agent.epsilon:.3f}')

            if i % 100 == 0 and i > 0:
                try:
                    agent.save_model()
                    print(f"Model saved at episode {i}")
                except Exception as e:
                    print(f"Error saving model: {e}")

        # Final model save
        try:
            agent.save_model()
            print("Final model saved")
        except Exception as e:
            print(f"Error saving final model: {e}")

        # After training loop, before plotting
        print("Saving game history...")
        pd.DataFrame(game_history).to_csv('/content/game_history.csv', index=False)
        print(f"Saved history for {len(game_history)} episodes")

        print("Training completed, creating plot...")
        filename = '/content/startups_plot.png'
        x = [i+1 for i in range(len(scores))]
        plotLearning(x, scores, eps_history, filename)
        print("Script completed successfully")

    except Exception as e:
        print(f"Error occurred: {e}")
        traceback.print_exc()
        sys.exit(1)

print("Script finished")

Streaming output truncated to the last 5000 lines.
Player 1 picks up from deck.
Player 1 puts down Octo Coffee putdown_shares.
The market is now {'Giraffe Beer': 1}
Player 1's shares are now: {'Octo Coffee': 2, 'Giraffe Beer': 1, 'Hippo Powertech': 1}
Player 1's anti-monopoly chips are now {'Hippo Powertech', 'Giraffe Beer', 'Octo Coffee'}
Player 2 picks up from deck.
Player 2 puts down Giraffe Beer putdown_shares.
The market is now {'Giraffe Beer': 1}
Player 2's shares are now: {'Elephant Mars Travel': 2, 'Giraffe Beer': 2}
Player 2's anti-monopoly chips are now {'Elephant Mars Travel'}
Player 3 picks up from deck.
Player 3 puts down Octo Coffee putdown_shares.
The market is now {'Giraffe Beer': 1}
Player 3's shares are now: {'Bowwow Games': 1, 'Flamingo Soft': 1, 'Hippo Powertech': 1, 'Octo Coffee': 1}
Player 3's anti-monopoly chips are now {'Bowwow Games', 'Flamingo Soft'}
RL agent's turn. Hand size: 3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1342330372096.0000
Action 13, Rewar

KeyboardInterrupt: 